#### **Install các thư viện cần thiết**

In [ ]:
!pip install scrapy
!pip install spider3

#### **Import packages**

In [1]:
import json
import re
import pandas as pd
import scrapy

#### **Tạo project với scrapy**

In [ ]:
!scrapy startproject laptopmedia

#### **Viết class để parse url của từng trang**

In [ ]:
import scrapy
import re 

class collect_laptop_url(scrapy.Spider):
  name='laptop_urls' 
  custom_settings = {
        'CONCURRENT_REQUESTS_PER_DOMAIN': 1,
        'DOWNLOAD_DELAY': 1
        }
  handle_httpstatus_list = [403]
  
  def start_requests(self):
    urls = ['https://laptopmedia.com/specs/?q=&hPP=1000&idx=laptops&p=0']
    yield scrapy.Request(url=urls[0], callback = self.parse)
  def parse(self, response):
    for href in response.xpath('//*[@id="results_container"]/div/div/article/a[1]/@href').extract():
        yield {
          'laptop_url': href
        }
    offset = re.findall(r'p=(\d+)', response.url)[0]
    if int(offset) < 20:
      next_url = 'https://laptopmedia.com/specs/?q=&hPP=20&idx=laptops&p=' + str(int(offset)+1) 
      yield response.follow(next_url, callback = self.parse, dont_filter=True)

In [ ]:
!pwd

In [ ]:
!cd laptopmedia/laptopmedia
!scrapy crawl laptop_urls -o dataset/laptop_urls.json

In [2]:
import requests
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from lxml import html
import json
import httpx
import urllib.request
from time import sleep



In [36]:
import pandas as pd
def url_generator(page_path, cat_id, page):
    return f"https://tiki.vn/api/personalish/v1/blocks/listings?limit=100&is_mweb=1&aggregations=2&sort=top_seller&urlKey={page_path}&category={cat_id}&page={page}"

page = httpx.get(url_generator('sach_truyen_tieng_viet', '316', '1')).json()
for i in page['filters']:
    if i['display_name'] == 'Danh Mục Sản Phẩm':
        categories = i['values']

print(categories[1])        
       
df = []
    
all_categories = []
for cat in categories:
    cat_data = httpx.get(url_generator(cat['url_key'], cat['query_value'], '1')).json()
    df.append(pd.DataFrame(cat_data['data']))
    has_child = False
    for i in cat_data['filters']: 
        if i['display_name'] == 'Danh Mục Sản Phẩm': # Nếu chứa danh mục con thì thêm vào
            #print(i['display_name'])
            all_categories.extend(i['values'])
            has_child = True
    
    if not has_child: 
        all_categories.append(cat)
        
print(len(all_categories))


{'count': 14195, 'display_value': 'Sách kinh tế', 'query_value': 846, 'url_key': 'sach-kinh-te'}
100


In [27]:
all_categories[0]

{'count': 198,
 'display_value': 'Du ký',
 'query_value': 4933,
 'url_key': 'du-ky'}

In [31]:
categories[0]

{'count': 27211,
 'display_value': 'Sách văn học',
 'query_value': 839,
 'url_key': 'sach-van-hoc'}

api limit = 100 -> Mỗi lần request lấy được 100 giá trị</br>
Cần lấy giá trị tối đa của số sách có thể có trong một thể loại 

In [61]:
all_categories[0]

{'count': 198,
 'display_value': 'Du ký',
 'query_value': 4933,
 'url_key': 'du-ky'}

**Lấy id của các cuốn sách ở từng thể loại**

In [58]:
# all_categories = pd.DataFrame(all_categories)
# display(all_categories)
def get_books_id(cat):
    n = (cat['count']//100) + 2
    books = {
        'Thể loại': cat['display_value'],
        'Book_id': []
    }
    for i in range(1, n):
        response = httpx.get(url_generator(cat['url_key'], cat['query_value'], i)).json()
        for book in response['data']:
            books['Book_id'].append({'id': book['id']})
    
    return books


In [59]:
for cate in all_categories:
    

{'Thể loại': 'Du ký',
 'Book_id': [{'id': 204317934},
  {'id': 193209826},
  {'id': 205279963},
  {'id': 3577373},
  {'id': 204649516},
  {'id': 548212},
  {'id': 113353587},
  {'id': 77160746},
  {'id': 205508265},
  {'id': 23943907},
  {'id': 135552449},
  {'id': 32314535},
  {'id': 205288789},
  {'id': 189588167},
  {'id': 206715176},
  {'id': 387725},
  {'id': 205293454},
  {'id': 644149},
  {'id': 14684956},
  {'id': 177315574},
  {'id': 129866864},
  {'id': 196902142},
  {'id': 109982962},
  {'id': 164044346},
  {'id': 26412706},
  {'id': 1256325},
  {'id': 111582626},
  {'id': 72882541},
  {'id': 159917640},
  {'id': 10190801},
  {'id': 193209819},
  {'id': 15867080},
  {'id': 62349850},
  {'id': 1058052},
  {'id': 48481757},
  {'id': 5995385},
  {'id': 192333128},
  {'id': 1667509},
  {'id': 15592527},
  {'id': 171580723},
  {'id': 197897214},
  {'id': 4708867},
  {'id': 127170159},
  {'id': 53720985},
  {'id': 8011013},
  {'id': 51550236},
  {'id': 71353257},
  {'id': 50214757